In [ ]:
import cv2
import os
import numpy as np
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import time
import numpy
 
# Importing functions and methods for OCR
from paddleocr.tools.infer.predict_rec import *
import paddleocr.tools.infer.utility as utility
from paddleocr.ppocr.postprocess import build_post_process
from paddleocr.ppocr.utils.logging import get_logger
from paddleocr.ppocr.utils.utility import get_image_file_list, check_and_read

In [ ]:
def rec(args, out_path, input, rec_model_dir,rec_image_shape = "3, 32, 320", rec_char_type = "ch", rec_algorithm = "CRNN", show = True, save = True):
    # Assigning values to args as the code is not running from the console.
    args.rec_model_dir = rec_model_dir
    args.rec_image_shape = rec_image_shape
    args.rec_char_type = rec_char_type
    args.rec_algorithm = rec_algorithm
    
 
    # Initializing some helper variables.
    t1 = 0
    t2 = 0
    tot = []
    os.chdir('./PaddleOCR')
 
    # Passing required values to the args variables.
    # if rec_algorithm == "SRN":
    #   args.rec_char_dict_path = 'ic15_dict.txt'
    #   args.use_space_char = False
    # 
    # if rec_algorithm == 'NRTR':
    #   args.rec_char_dict_path = './ppocr/utils/EN_symbol_dict.txt'
    #   args.rec_image_shape = "1,32,100"
    
    # Initializing recognizer.
    image_file_list = get_image_file_list(input)
    text_recognizer = TextRecognizer(args)
    valid_image_file_list = []
    img_list = []
 
    # Warming up the GPU to run it at its full capacity.
    if args.warmup:
        image = np.random.uniform(0, 255, [32, 320, 3]).astype(np.uint8)
        for i in range(10):
            res = text_recognizer([image])
 
    # Reading and appending all images' array to a list.
    for image_file in image_file_list:
        image, flag = check_and_read(image_file)
        if not flag:
            image = cv2.imread(image_file)
        if image is None:
            logger.info("error in loading image:{}".format(image_file))
            continue
        valid_image_file_list.append(image_file)
        img_list.append(image)
 
    # Applying OCR to the images
    t1 = time.time()
    try:
        rec_res, _ = text_recognizer(img_list)
    except Exception as E:
        logger.info(traceback.format_exc())
        logger.info(E)
        exit()
    
    # Calculating FPS and printing the info.
    t2 = time.time()
    fps = str(t2-t1)
    for ino in range(len(img_list)):
        logger.info("Predicts of {}:{}".format(valid_image_file_list[ino],
                                              rec_res[ino]))
        if save:
          cv2.imwrite(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '_rec' + '.jpg'),img_list[ino])
          with open(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '.txt'), 'w') as f:
            f.write(str(rec_res[ino]))
    logger.info("Time taken recognize all images : {}".format(fps))
    print(len(image_file_list))
    logger.info("Average fps : {}".format(1/(float(fps)/len(image_file_list))))
    
    #cv2.putText(image, str(rec_res[ino]),(0, int(15 * 1)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), thickness=1)
    
    # Displaying and saving the output according to the parameters set.
    if show:
      plt.figure(figsize = (25,14))
      plt.imshow(image)
      plt.show()

In [ ]:
def score_calc(pth, annot):
  # Importing distance metric.
  from Levenshtein import distance
  score_all = []
 
  # Looping through the output text files and storing the OCR output in a variable.
  for out_file in os.listdir(pth):
    if out_file.endswith('.txt'):
      with open(os.path.join(pth, out_file), 'rb') as f:
        out = f.read()
        f.close()
      
      # Cleaning the OCR output text.
      try:
        out = str(out).split(',')[1].split(',')[0].replace("'", '').lower()
      except:
        print('OCR output does not exist')
 
      # Opening the ground truth file and calculating weighted distance between ground truth and OCR output.
      with open(annot) as f:
        for line in f:
          #print(out_file.split('.')[0], line.split(',')[0])
          if line.split(',')[0] == out_file.split('.')[0]:
            gt = line.split(',')[1].lower()
            score = distance(str(out), str(gt))/len(gt)
            score_all.append(score)
            break
  # Printing the average score.
  print("final score:", sum(score_all)/len(score_all))

In [ ]:
out_path = 'out_def'
rec_model_dir = 'rec_dir'
input = 'input_dir'
sys.argv = ['']
rec(utility.parse_args(), out_path, input, rec_model_dir, show = False)

____________________NEXT____________________

In [ ]:
from paddleocr import PaddleOCR

# Specify the path to the non-default detection model
# det_model_dir = 'det_dir'
# rec_model_dir = 'rec_dir'
# rec_char_dict_path=''

# Initialize the OCR class with the det_model_dir parameter
ocr = PaddleOCR(lang='ru')

# Use the OCR instance for text detection
result = ocr.ocr('input_dir/2.png')
print(result)

In [65]:
from paddleocr import PaddleOCR,draw_ocr
import os
import cv2
import matplotlib.pyplot as plt

image = None

def save_ocr(img_path, out_path, result, font):
  global img
    
  save_path = out_path
  print(len(*result))
  print(result)
  image = cv2.imread(img_path)

      
  boxes = [line[-1][0] for line in result]
    
  txts = [line[-1][1][0] for line in result]
 
  image = draw_ocr(image, boxes, txts, font_path=font)
  
  cv2.imwrite(save_path, image)
 
  img = cv2.cvtColor(image)
  plt.imshow(img)

In [66]:
out_path = 'out_def/out_2.png'
img_path = 'input_dir/3.jpg'
ocr = PaddleOCR(lang='ru')
result = ocr.ocr(img_path)

[2024/04/13 12:20:08] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Andre/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Andre/.paddleocr/whl\\rec\\cyrillic\\cyrillic_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_bat

In [67]:
font = 'simfang.ttf'
save_ocr(img_path, out_path, result, font)

33
[[[[[1707.0, 122.0], [1843.0, 110.0], [1850.0, 191.0], [1715.0, 204.0]], ('оо', 0.8622561693191528)], [[[1852.0, 133.0], [1924.0, 133.0], [1924.0, 185.0], [1852.0, 185.0]], ('г.', 0.7508884072303772)], [[[2538.0, 136.0], [2970.0, 113.0], [2981.0, 328.0], [2549.0, 351.0]], ('оо', 0.9191939234733582)], [[[1569.0, 148.0], [1669.0, 136.0], [1678.0, 204.0], [1577.0, 217.0]], ('нд', 0.9174261093139648)], [[[1186.0, 166.0], [1367.0, 143.0], [1376.0, 221.0], [1195.0, 243.0]], ('цеh', 0.7657256722450256)], [[[856.0, 177.0], [1174.0, 167.0], [1177.0, 244.0], [858.0, 254.0]], ('ишеваа', 0.9117700457572937)], [[[1734.0, 293.0], [2103.0, 283.0], [2105.0, 343.0], [1736.0, 353.0]], ('SHEPгETичECKAя', 0.7607287764549255)], [[[1508.0, 335.0], [1634.0, 335.0], [1634.0, 382.0], [1508.0, 382.0]], ('зOAы', 0.5620860457420349)], [[[1121.0, 352.0], [1289.0, 352.0], [1289.0, 412.0], [1121.0, 412.0]], ('жиры', 0.6446717977523804)], [[[1801.0, 344.0], [2035.0, 334.0], [2038.0, 394.0], [1803.0, 405.0]], ('цEн

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - cvtColor() missing required argument 'code' (pos 2)
>  - cvtColor() missing required argument 'code' (pos 2)
